In [2]:
import pandas as pd
import numpy as np

import nltk
from nltk.tokenize import word_tokenize
from nltk import PorterStemmer
from nltk import WordNetLemmatizer
from nltk.corpus import stopwords

import re

from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.svm import SVR

In [4]:
cd

/Users/brianrochford


In [9]:
df = pd.read_csv('winemag_fin.csv', index_col = 0)

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [10]:
df.describe()

,points,price
count,280901.000000,258210.000000
mean,88.146934,34.177162
std,3.151528,38.611708
min,80.000000,4.000000
25%,86.000000,16.000000
50%,88.000000,25.000000
75%,90.000000,40.000000
max,100.000000,3300.000000


In [8]:
df = df.drop_duplicates()
df = df.drop(columns=['Unnamed: 0','designation','region_1','region_2','province','taster_name','taster_twitter_handle','title','variety','winery'])

KeyError: "['Unnamed: 0'] not found in axis"

In [119]:
df = df.dropna()

In [12]:
df_noDups = df.drop_duplicates()[['description','region_1','points','price']].dropna()

In [104]:
df = df.drop_duplicates()
df= df.dropna()

In [105]:
df.region_1.value_counts()

Columbia Valley (WA)         2281
Willamette Valley            1253
Russian River Valley         1175
Napa Valley                  1046
Paso Robles                   838
                             ... 
Sierra Pelona Valley            1
York Mountain                   1
Sonoma-Mendocino                1
Sonoma County-Lake County       1
San Diego County                1
Name: region_1, Length: 171, dtype: int64

In [13]:
data1 = pd.DataFrame(df.region_1.value_counts())
data1.columns = ['counts'] 
data1

,counts
Napa Valley,10689
Columbia Valley (WA),9099
Russian River Valley,6662
California,6091
Mendoza,5887
Paso Robles,5403
Willamette Valley,4397
Alsace,3737
Rioja,3362
Brunello di Montalcino,3216


In [14]:
data2 = df.groupby('region_1').agg({'description':lambda x: ' '.join(x), 
                         'points':'mean', 
                         'price':'mean'})

In [16]:
df_joinCountRegion = data1.join(data2)

In [20]:
df_joinCountRegion

,counts,description,points,price
Napa Valley,10689,"Soft, supple plum envelopes an oaky structure ...",88.757414,51.897244
Columbia Valley (WA),9099,"Aromas of cranberry, barrel spice and herb are...",88.673481,27.418674
Russian River Valley,6662,This wine is put together from multiple vineya...,89.171720,39.247322
California,6091,"Very deep in color and spicy-smoky in flavor, ...",85.187654,15.419747
Mendoza,5887,Raw black-cherry aromas are direct and simple ...,86.131136,20.730749
Paso Robles,5403,This wine from the Geneseo district offers aro...,86.917453,31.910175
Willamette Valley,4397,"Tart and snappy, the flavors of lime flesh and...",88.796225,33.925588
Alsace,3737,This dry and restrained wine offers spice in p...,89.120417,31.789424
Rioja,3362,"Slightly foxy on the nose, with rubbery aromas...",87.445866,32.471381
Brunello di Montalcino,3216,"Intense aromas of ripe red berry, menthol, esp...",90.601057,73.498013


In [19]:
df_joinCountRegion.to_excel('regionCountMerge.xlsx')

In [21]:
df.region_1.value_counts()[:350]

Napa Valley                                           10689
Columbia Valley (WA)                                   9099
Russian River Valley                                   6662
California                                             6091
Mendoza                                                5887
Paso Robles                                            5403
Willamette Valley                                      4397
Alsace                                                 3737
Rioja                                                  3362
Brunello di Montalcino                                 3216
Sonoma County                                          3099
Toscana                                                3082
Barolo                                                 2997
Champagne                                              2982
Sonoma Coast                                           2947
Finger Lakes                                           2937
Sicilia                                 

In [22]:
remove_non_alphabets = lambda x: re.sub(r'[^a-zA-Z]',' ',x)

tokenize = lambda x: word_tokenize(x)

ps = PorterStemmer()

stem = lambda w: [ ps.stem(x) for x in w ]

lemmatizer = WordNetLemmatizer()
leammtizer = lambda x: [ lemmatizer.lemmatize(word) for word in x ]

remove_stop = lambda x: [w for w in x if w not in stop]

In [42]:
#df_joinCountRegion

In [37]:

#df.region_1.value_counts()[:350]
df_joinCountRegion2 = df_joinCountRegion.iloc[0].value_counts()[:350]
df_joinCountRegion3 = pd.DataFrame(df_joinCountRegion2)
#df_joinCountRegion3

In [38]:
df = df_joinCountRegion3
df.describe()

,Napa Valley
count,4.0
mean,1.0
std,0.0
min,1.0
25%,1.0
50%,1.0
75%,1.0
max,1.0


In [40]:
#df

In [41]:
print('Processing : [=', end='')
df['description'] = df['description'].apply(remove_non_alphabets)
print('=', end='')
df['description'] = df['description'].apply(tokenize)
print('=', end='')
stop = text.ENGLISH_STOP_WORDS
df['description'] = df['description'].apply(remove_stop)
print('=', end='')
df['description'] = df['description'].apply(stem)
print('=', end='')
df['description'] = df['description'].apply(leammtizer)
print('=', end='')
df['description'] = df['description'].apply(lambda x: ' '.join(x))
print(']')

Processing : [=

KeyError: 'description'

In [44]:
cd


/Users/brianrochford


In [45]:
df = pd.read_csv('preprocessedwine.csv', index_col= 0)
df = df.dropna()

In [52]:
df.columns

Index(['country', 'description', 'points', 'price'], dtype='object')

In [46]:
Q1 = np.percentile(df['price'], 25,
                   interpolation = 'midpoint')
 
Q3 = np.percentile(df['price'], 75,
                   interpolation = 'midpoint')
IQR = Q3 - Q1
df = df[df.price < (Q3+1.5*IQR)+1]

In [47]:
def get_sample(df,x):
    df = df.sample(frac=x, replace=False, random_state=1)
    df = df.reset_index(drop=True)
    return df

def bow_vector(df):
    bow_vectorizer=CountVectorizer(min_df=1, max_features=5000, ngram_range=(1,1))

    bow_features = bow_vectorizer.fit_transform(df.description)

    df_bow = pd.DataFrame(bow_features.toarray(),
                      columns=bow_vectorizer.get_feature_names())
    df_bow = df_bow.drop(columns=['price'])
    
    return df_bow

def jointwo(df,df_bow):
    df = df.join(df_bow)
    df = df.drop(columns=['country','description'])
    df = df.fillna(0)
    return df

In [48]:
def run_random_for(df):
    X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['price']), df.price, test_size=0.30, random_state=42)
    regr = RandomForestRegressor()
    regr.fit(X_train, y_train)
    return regr, X_test, y_test
    
def run_svr(df):
    X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['price']), df.price, test_size=0.30, random_state=42)
    regressor = SVR(kernel='rbf')
    regressor.fit(X_train,y_train)
    return regressor, X_test, y_test

In [49]:
df1 = get_sample(df,0.1)
df1 = df1.head(int(len(df)*(100/100)))
df_bow = bow_vector(df1)
df2 = jointwo(df1,df_bow)

In [54]:
df2

,points,price,abbrevi,abil,abl,abli,abound,about,abras,abrupt,...,zesti,zin,zinfandel,zing,zingi,zip,zippi,zone,zonin,zweigelt
0,89,18.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,92,35.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,80,16.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,87,19.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,86,15.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25810,86,20.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25811,87,14.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25812,85,23.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25813,88,12.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [53]:
df.region_1.value_counts()

AttributeError: 'DataFrame' object has no attribute 'region_1'

In [54]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

test = SelectKBest(score_func=chi2, k=4)
fit = test.fit(df2.drop(columns=['price']), df2.price)


np.set_printoptions(precision=3)
print(fit.scores_)

features = fit.transform(df2.drop(columns=['price']))

print(features[0:5,:])

[798.326  16.311  97.357 ...  39.8    20.113  43.687]
[[0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 1]
 [0 0 0 0]]


In [56]:
regr1, x_test1, y_test1 = run_random_for(df2)

In [55]:
fet_imp = pd.DataFrame({"names":x_test1.columns,
                       "importance":regr1.feature_importances_})
best_features = fet_imp.sort_values(by=['importance'],ascending =False).names[:100]

NameError: name 'x_test1' is not defined

In [ ]:
df3 = df1.head(int(len(df)*(30/100)))
df_bow1 = bow_vector(df3)
df_bow1 = df_bow1.filter(items=list(best_features))
df4 = jointwo(df3,df_bow1)

In [ ]:
regr2, x_test2, y_test2 = run_random_for(df4)
y_pred = regr2.predict(x_test2)

In [ ]:
mean_squared_error(y_test2, y_pred)